In [ ]:
import numpy as np
import pandas as pd

'''
q-learner
    -policy
    -value function
    -update()
    -move()
    
portfolio
    -cash
    -assets
    -orderHistory
        -orders[{time, asset, action, amount, value}]
        -show()
    -show()
    
'''